# <center>INFO MSO 3.2 - Représentation et manipulation de données structurées</center>

## <center style="color: #66d">BE - Application XML, Import, Validation, Transformation, Impression</center>

### Déroulement de la séance

Ce travail s'effectue par binômes. Merci de vous organiser et de prendre les coordonnées de votre binôme pour pouvoir finir si nécessaire le travail après la séance.

Les livrables qui vous seront demandés pour ce BE devront faire l'objet d'un UNIQUE fichier zippé (.zip, .gz, .tgz, .tar, .7z - les formats exotiques non listés ici ne seront pas pris en compte) et déposés sur le serveur moodle dans la zone "Bureau d'Etude".

Le compte-rendu est à déposer <a href="https://pedagogie3.ec-lyon.fr/mod/assign/view.php?id=36070">[ici]</a> par l'un OU l'autre membre du binôme sur le serveur moodle dans la rubrique <i>Bureau d'Etude</i> du cours <i>INFO MSO 3.2 - Représentation et manipulation de données structurées</i>. 

Au moment du dépôt, merci de bien vouloir rédiger le champ Titre sous la forme :

          CR <nom eleve 1> - <nom eleve 2>  

### 1. Définition de type d'un document XML

Avec le sujet, vous avez trouvé un fichier de données nommé <tt>regularite-mensuelle-tgv.csv</tt> contenant des informations sur la régularité des TGV entre septembre 2011 et juin 2015.

Vous pouvez observer la nature et la structure des informations que contient ce fichier à l'aide de votre tableur préféré, ou à défaut avec un éditeur de texte.

L'objectif est de créer un document XML correspondant à une _application XML_ permettant de stocker ces mêmes données, en évitant au maximum la redondance d'information. Cette application est décrite par la DTD suivante :

<b>Q1. Modèle de document XML :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Donner un exemple de document XML le plus simple possible correspondant à la structure préconisée par cette DTD
    (c'est-à-dire avec un seul élément correspondant à chacun des contenus obligatoires).
</div>

### 2. Validation d'un document XML

Il existe un certain nombre de sites en ligne permettant la validation d'un document XML à l'aide d'une DTD. Une simple recherche avec les mots-clés <i>XML validator</i> pourra vous en convaincre.
Il est toutefois beaucoup plus pratique de valider ses documents à l'aide de quelques lignes de code maison, comme il vous est proposé ci-dessous.

<b>Q2. Validation du document d'exemple :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px"><ul>
<li>Inclure au document simplifié que vous venez de définir la DTD fournie sous forme de DTD interne,</li>
    <li>enregistrer ce document sous le nom <a href="question-2.xml"><tt>question-2.xml</tt></a>,</li>
    <li>puis valider le document obtenu à l'aide d'un programme python que vous développerez à l'aide du module <a href="http://lxml.de/validation.html"><tt>lxml</tt></a>.</li>
    </ul>
</div>

In [1]:
from lxml import etree


def validate_tag(element, expected_tag: str) -> bool:
    if element.tag != expected_tag:
        return False
    return True

def validate_attribs(element, mandatory_attribs: list[str] = [], optional_attribs: list[str] = []) -> bool:
    attribs = element.attrib

    for key in mandatory_attribs:
        if key not in attribs:
            return False
        
    for key in attribs:
        if key not in mandatory_attribs and key not in optional_attribs:
            return False
    
    return True


def validate(filename):
    try:
        with open(filename, 'r') as f:
            xml = f.read()
            regularite_tgv = etree.fromstring(xml)
    except etree.XMLSyntaxError as e:
        print(e)
        return False
    
    if not validate_tag(regularite_tgv, 'regularite-tgv'):
        return False
    if not validate_attribs(regularite_tgv):
        return False
    
    for axe in regularite_tgv:
        if not validate_tag(axe, 'axe'):
            return False
        
        if not validate_attribs(axe, ['nom']):
            return False
        
        for gare_depart in axe:
            if not validate_tag(gare_depart, 'gare-depart'):
                return False
            
            if not validate_attribs(gare_depart, ['nom']):
                return False
            
            for gare_arrivee in gare_depart:
                if not validate_tag(gare_arrivee, 'gare-arrivee'):
                    return False
                
                if not validate_attribs(gare_arrivee, ['nom']):
                    return False
                
                for mesure in gare_arrivee:
                    if not validate_tag(mesure, 'mesure'):
                        return False
                    
                    if not validate_attribs(mesure, [        
                            'annee',
                            'mois',
                            'trains-prevus',
                            'trains-ok',
                            'annules',
                            'retards',
                            'regularite',
                            'commentaire',
                        ]
                    ):
                        return False
                    
                    if len(mesure) != 0:
                        return False

    return True

validate('question-2.xml')

ValueError: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.

### 3. Création d'un document XML

<b>Q3. Import :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
En vous reportant au mode d'emploi du module python nommé <a href="https://docs.python.org/3/library/csv.html"><tt>csv</tt></a> pour la lecture du fichier de données, et <a href="http://lxml.de/tutorial.html"><tt>lxml.etree</tt></a> pour la création de l'arbre XML, compléter ci-dessous le programme python de manière à créer un document XML conforme à la DTD préconisée et contenant les données du fichier CSV fourni. Exécuter ce programme pour créer le document XML que vous nommerez 
    <a href="question-3.xml"><tt>question-3.xml</tt></a>.
</div>

In [2]:
# Génération du document XML 
from lxml import etree
import csv

# Lecture des informations
axes = {}
with open("regularite-mensuelle-tgv.csv",'r',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=';', quotechar='"')
    header = next(reader)
    entry = {}
    for row in reader:
        annee, mois = row[0].split('-')
        axe = row[1]
        gare_depart = row[2]
        gare_arrivee = row[3]
        trains_prevus = row[4]
        trains_ok = row[5]
        annules = row[6]
        retards = row[7]
        regularite = row[8]
        commentaire = row[9]

        existing_axe = axes.get(axe, {}) 
        existing_gare_depart = existing_axe.get(gare_depart, {})
        existing_gare_arrivee = existing_gare_depart.get(gare_arrivee, [])

        mesure = {
            'annee': annee,
            'mois': mois,
            'trains-prevus': trains_prevus,
            'trains-ok': trains_ok,
            'annules': annules,
            'retards': retards,
            'regularite': regularite,
            'commentaire': commentaire,
        }

        existing_gare_arrivee.append(mesure)

        existing_gare_depart[gare_arrivee] = existing_gare_arrivee

        existing_axe[gare_depart] = existing_gare_depart

        axes[axe] = existing_axe

# création de l'arbre XML en mémoire
root = etree.fromstring('<regularite-tgv/>')
for a in axes:
    axe = axes[a]
    axe_element = etree.SubElement(root, 'axe', nom=a)
    
    for gare_depart in axe:
        gare_depart_element = etree.SubElement(axe_element, 'gare-depart', nom=gare_depart)
        for gare_arrivee in axe[gare_depart]:
            gare_arrivee_element = etree.SubElement(gare_depart_element, 'gare-arrivee', nom=gare_arrivee)
            for mesure in axe[gare_depart][gare_arrivee]:
                mesure_element = etree.SubElement(gare_arrivee_element, 'mesure', mesure)

# sérialisation dans un fichier
with open('question-3.xml', 'wb') as f:
    f.write(etree.tostring(root.getroottree(), pretty_print=True, encoding='utf-8'))

<b>Q4. Validation du document XML :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Comme pour le document simple inclure de la même manière que précédemment la DTD fournie sous forme de DTD interne au document généré, puis valider ce document.
</div>
<p><b>N.B.</b> Le document comportant la DTD interne pourra être nommé 
    <a href="question-4.xml"><tt>question-4.xml</tt></a>.</p>

In [3]:
# validation du document XML obtenu
validate('question-4.xml')

True

### 4. Transformation du document XML

On se propose maintenant de transformer ce document XML à l'aide de XSLT afin de pouvoir visualiser une partie des informations au format HTML ou XHTML.

<b>Q5. Développer une première feuille de style</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Dupliquer le document XML en l'appelant question-5.xml, puis ajouter un lien vers une feuille de style XSL nommée question-5.xsl que vous développerez.
L'objectif est d'afficher dans un navigateur le document XML via une feuille de style XSL, avec les informations disposées sous forme de tables, en veillant notamment à afficher la ponctualité avec un chiffre significatif après la virgule, et un seul.
</div>

<div style="margin-top:0.5em;background-color:#fee;padding:10px;border-radius:3px">
Afin de pouvoir visualiser le résultat d'un document XML transformé par XSL par un navigateur, il est nécessaire pour des raisons de sécurité que les documents soient délivrés via un serveur et non pas directement ouverts depuis le disque local. Le plus simple pour cela est de démarrer un serveur via python, à l'aide de la commande ci-dessous exécutée depuis une fenêtre terminal dans le répertoire de votre BE :
<code>python -m http.server</code>

Lorsque vous aurez développé la feuille de style correspondante; ceci vous permettra de visualiser votre document à l'adresse : <a href="http://localhost:8000/question-5.xml"><tt>http://localhost:8000/question-5.xml</tt></a>
</div>

<p>
<b>N.B.</b> L'obtention d'un document HTML ou XHTML via une transformation XSL n'empêche pas que celui-ci comporte une feuille de style CSS locale, ou fasse appel à une feuille de style CSS externe nommée par exemple 
    <a href="question-5.css"><tt>question-5.css</tt></a>
    pour son affichage par le navigateur. On pourra appeler la feuille XSLT correspondante
    <a href="question-5-css.xsl"><tt>question-5-css.xsl</tt></a>
    et le document XML y faisant appel
    <a href="http://localhost:8000/question-5-css.xml"><tt>question-5-css.xml</tt></a>.
</p>

<b>Q6. Afficher les mois par leur nom et non pas par leur numéro :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créeer un template paramétré pour convertir un numéro de mois en nom de mois en toutes lettres. La nouvelle version de la feuille de style s'appellera <a href="question-6.xsl"><tt>question-6.xsl</tt></a>
</div>
<div style="margin-top:0.5em">
    Voir le résultat : <a href="http://localhost:8000/question-6.xml"><tt>question-6.xml</tt></a>
</div>

<b>Q7. Afficher les dates limites dans le titre de la page :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Créer des templates paramétrés pour calculer le minimum (resp. le maximum) d'une liste. Utiliser ces templates depuis
    la feuille de style <a href="question-7.xsl"><tt>question-7.xsl</tt></a>
    pour afficher un titre du type "Régularité des TGV de septembre 2011 à juin 2015" où les dates sont automatiquement extraites du document XML.
</div>
<div style="margin-top:0.5em">
Voir le résultat : <a href="http://localhost:8000/question-7.xml">question-7.xml</a>
    </div>

### 5. Prise en main de XSL-FO et installation de Apache fop

Afin de pouvoir transformer le document XML de données pour obtenir d'abord un document XSL-FO, puis un rapport papier au format <tt>pdf</tt> il sera nécessaire d'installer l'outil "fop" (Apache Formatting Objects Processor).

<b>Q8. Prise en main de XSL-FO :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Consulter le <a href="http://dmolinarius.github.io/demofiles/mod-84/xslfo.pdf">cours sur XSL-FO</a>, puis
créer une feuille de style XSLT nommée <a href="question-8.xsl"><tt>question-8.xsl</tt></a> qui transforme n'importe quel document XML en un document XSL-FO (toujours le même, quel que soit le contenu du document XML transformé) contenant le message "hello, world".
</div>

<b>Q9. Créer un premier document XSL-FO :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
La transformation XSLT d'un document XML via votre feuille de style peut être très simplement implémentée à l'aide du module lxml ce qui permettra d'admirer le résultat obtenu dans un fichier qui sera nommé <tt>question-9.fo</tt>
</div>

In [4]:
from lxml import etree

# document xml vide
xml = etree.XML("<root></root>")

# lecture de la feuille de style xsl
xsl = etree.parse("question-8.xsl")

# transformation du document xml par la feuille de style
transform = etree.XSLT(xsl)
result = transform(xml)

# sérialisation dans un fichier
with open('question-9.fo', 'wb') as f:
    f.write(etree.tostring(result, pretty_print=True, encoding='utf-8'))

In [5]:
# voir le document question-9.fo
%pycat question-9.fo

<fo:root xmlns:fo="http://www.w3.org/1999/XSL/Format">
  <fo:layout-master-set>
    <fo:simple-page-master master-name="page-unique" page-height="29.7cm" page-width="21cm" margin-top="1.5cm" margin-bottom="2cm" margin-left="2.5cm" margin-right="1cm">
      <fo:region-body background-color="#CCCCCC"/>
    </fo:simple-page-master>
  </fo:layout-master-set>
  <fo:page-sequence master-reference="page-unique">
    <fo:flow flow-name="xsl-region-body">
      <fo:block>Hello World !</fo:block>
    </fo:flow>
  </fo:page-sequence>
</fo:root>


<b>Q10. Installer fop et obtenir un document pdf :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Installer <a href="https://xmlgraphics.apache.org/fop/">Apache FOP</a> et transformer le document <tt>question-9.fo</tt> pour obtenir <a href="question-10.pdf"><tt>question-10.pdf</tt></a>.
</div>

Exemple de fonction réalisant la transformation depuis le notebook :

In [3]:
# fonction pour lancer la transformation fop
def run_fop(*fop_args):
    import subprocess
    
    # appel de fop (ajuster éventuellement le chemin d'accès à l'exécutable et au fichier de configuration)
    args = ["fop", *fop_args]
    r = subprocess.run(args,shell=False,stderr=subprocess.PIPE)
    return r.stderr.decode('iso-8859-1')

In [7]:
# On effectue la transformation
r = run_fop('question-9.fo', 'question-10.pdf')
print(r)

Mar 25, 2024 1:05:07 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #1.



In [8]:
# Méthode alternative pour lancer fop
!fop question-9.fo question-10.pdf

Mar 25, 2024 1:05:07 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #1.


<b>N.B.</b> Il est utile de mentionner ici que fop est non seulement capable d'obtenir comme ci-dessus un document pdf à partir d'un document XSL-FO, mais également d'effectuer au préalable la transformation d'un document xml via xslt avec une feuille de style xsl. Voir pour cela les options de la ligne de commande permettant de lancer fop.  

In [9]:
!fop --help

/usr/bin/fop [script options] [FOP options]
Script Options:
  --help, -h             print this message and FOP help
  --noconfig             suppress sourcing of /etc/fop.conf,
                         $HOME/.fop/fop.conf, and $HOME/.foprc
                         configuration files
  --execdebug            print FOP exec line generated by this
                         launch script
FOP Version 2.9

USAGE
fop [options] [-fo|-xml] infile [-xsl file] [-awt|-pdf|-mif|-rtf|-tiff|-png|-pcl|-ps|-txt|-at [mime]|-print] <outfile>
 [OPTIONS]  
  -version          print FOP version and exit
  -x                dump configuration settings  
  -c cfg.xml        use additional configuration file cfg.xml
  -l lang           the language to use for user information 
  -nocs             disable complex script features
  -r                relaxed/less strict validation (where available)
  -dpi xxx          target resolution in dots per inch (dpi) where xxx is a number
  -s                for area tre

### 6. Création d'un rapport

Maintenant que la chaîne de production est maîtrisée, l'objectif sera de créer un rapport au format pdf, à partir des données du document de base <a href="question-3.xml"><tt>question-3.xml</tt></a> qui servira de source pour toute la suite du BE.

<b>Q11. Liste des axes :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Développer une feuille de style XSL qui transforme ce document de manière à obtenir un document <tt>pdf</tt> listant le nom des axes sous forme de titres.
</div>

On nommera les documents demandés
<a href="question-11.xsl"><tt>question-11.xsl</tt></a> et
<a href="question-11.pdf"><tt>question-11.pdf</tt></a>.

In [10]:
# Transformation pour obtention du document pdf
r = run_fop('-xml', 'question-3.xml', '-xsl', 'question-11.xsl', '-pdf', 'question-11.pdf')
print(r)

Mar 25, 2024 1:05:10 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #1.
Mar 25, 2024 1:05:10 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #2.
Mar 25, 2024 1:05:10 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #3.
Mar 25, 2024 1:05:10 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #4.



<b>Q12. Page de titre :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Ajouter une page de titre. En profiter pour y mentionner vos noms et la date d'édition en passant des variables appropriées au moteur XSLT via la ligne de commande de fop.
</div>

Les documents demandés seront nommés
<a href="question-12.xsl"><tt>question-12.xsl</tt></a> et
<a href="question-12.pdf"><tt>question-12.pdf</tt></a>.

In [1]:
def run_transform(args):
    from datetime import datetime

    # date de la transformation
    d = datetime.now()
    date_components = [
        ['lundi','mardi','mercredi','jeudi','vendredi','samedi','dimanche'][d.weekday()],
        d.day,
        ['janvier', 'février', 'mars', 'avril', 'mai','juin', 'juillet',
         'août', 'septembre', 'octobre', 'novembre', 'décembre'][d.month-1],
        d.year
    ]
    date="{} {} {} {}".format(*date_components)

    # Transformation pour obtention du document pdf
    fop_args = [
        '-param', 'date', date,
        '-param', 'hour', str(d.hour),
        '-param', 'minutes', str(d.minute),
        '-param', 'author', 'Matías Duhalde',
        *args
    ]
    return run_fop(*fop_args)

In [9]:
r = run_transform([
    '-xml', 'question-3.xml',
    '-xsl', 'question-12.xsl',
    '-pdf', 'question-12.pdf'
])
print(r)

Mar 25, 2024 1:22:04 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:22:04 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:22:04 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #1.
Mar 25, 2024 1:22:04 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #2.
Mar 25, 2024 1:22:04 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #3.
Mar 25, 2024 1:22:04 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #4.
Mar 25, 2024 1:22:04 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #5.



<b>Q13. Statistiques :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Pour chacun des couples gare de départ / gare d'arrivée, créer un tableau avec les statistiques de la ligne, en évitant que les tableaux puissent être coupés par un saut de page. Ne pas oublier d'afficher les statistiques dans l'ordre du calendrier, et non pas dans l'ordre du document source...
</div>
<p>
Les documents demandés seront nommés
<a href="question-13.xsl"><tt>question-13.xsl</tt></a>,
<a href="question-13.pdf"><tt>question-13.pdf</tt></a>.
</p>

In [13]:
r = run_transform(['-xml','question-3.xml', '-xsl','question-13.xsl', '-pdf','question-13.pdf'])
print(r)

Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
INFO: Rendered page #1.
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 2024 1:26:07 PM org.apache.fop.apps.FOUserAgent processEvent
Mar 25, 20

<b>Q14. Commentaires :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Faire figurer les commentaires non vides là aussi dans l'ordre du calendrier.
</div>

Les documents demandés seront nommés
<a href="question-14.xsl"><tt>question-14.xsl</tt></a> et
<a href="question-14.pdf"><tt>question-14.pdf</tt></a>.

In [69]:
r = run_transform(['-xml','question-3.xml', '-xsl','question-14.xsl', '-pdf','question-14.pdf'])
print(r)


USAGE
fop [options] [-fo|-xml] infile [-xsl file] [-awt|-pdf|-mif|-rtf|-tiff|-png|-pcl|-ps|-txt|-at [mime]|-print] <outfile>
 [OPTIONS]  
  -version          print FOP version and exit
  -x                dump configuration settings  
  -c cfg.xml        use additional configuration file cfg.xml
  -l lang           the language to use for user information 
  -nocs             disable complex script features
  -r                relaxed/less strict validation (where available)
  -dpi xxx          target resolution in dots per inch (dpi) where xxx is a number
  -s                for area tree XML, down to block areas only
  -v                run in verbose mode (currently simply print FOP version and continue)

  -o [password]     PDF file will be encrypted with option owner password
  -u [password]     PDF file will be encrypted with option user password
  -noprint          PDF file will be encrypted without printing permission
  -nocopy           PDF file will be encrypted without copy

<b>Q15. Améliorer <i>ad libitum</i> :</b>
<div style="background-color:#eef;padding:10px;border-radius:3px">
Numéroter les pages, ajouter un graphique SVG par couple gare de départ / gare d'arrivée,
créer un sommaire cliquable renvoyant directement à la page concernée...
</div>

Les documents demandés seront nommés
<a href="question-15.xsl"><tt>question-15.xsl</tt></a> et
<a href="question-15.pdf"><tt>question-15.pdf</tt></a>.

In [73]:
r = run_transform(['-xml','question-3.xml', '-xsl','question-15.xsl', '-pdf','question-15.pdf'])
print(r)

mars 20, 2024 9:37:52 PM org.apache.fop.apps.FopConfParser configure
INFOS: Default page-height set to: 11.00in
mars 20, 2024 9:37:52 PM org.apache.fop.apps.FopConfParser configure
INFOS: Default page-width set to: 8.50in
mars 20, 2024 9:38:01 PM org.apache.fop.events.LoggingEventListener processEvent
INFOS: Rendered page #1.
mars 20, 2024 9:38:01 PM org.apache.fop.events.LoggingEventListener processEvent
AVERTISSEMENT: Font "Symbol,normal,700" not found. Substituting with "Symbol,normal,400".
mars 20, 2024 9:38:01 PM org.apache.fop.events.LoggingEventListener processEvent
AVERTISSEMENT: Font "ZapfDingbats,normal,700" not found. Substituting with "ZapfDingbats,normal,400".
mars 20, 2024 9:38:02 PM org.apache.fop.events.LoggingEventListener processEvent
AVERTISSEMENT: Font "Franklin Gothic Medium,italic,700" not found. Substituting with "Franklin Gothic Medium,italic,500".
mars 20, 2024 9:38:08 PM org.apache.fop.events.LoggingEventListener processEvent
INFOS: Rendered page #2.
mars 20, 